In [1]:
import os
import re
import traceback

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import pickle
import time
import numpy as np

In [3]:
from unitils import get_ans,ans_clean,pre_clean,f1
from one_jump import one_jump_solver
from two_jump import two_jump_solver,down_to_low
from config import df,stop_words

In [33]:
from unitils import from_entry,match_type_cur,get_entry,from_value,jaccard,hint,local_les,get_type

In [5]:
SYNONYM_MAP = {
    '丈夫': '丈夫配偶',
    '老公': '丈夫配偶',
    '妻子': '妻子配偶',
    '夫人': '夫人妻子配偶',
    '媳妇': '妻子配偶',
    '老家': '老家籍贯',
    '故乡': '故乡籍贯',
    '家乡': '家乡籍贯', 
    '英文':'外文英文',
    '西班牙语':'外文西班牙语',
    '贡献' : '贡献主要成就',
    '功绩' : '功绩主要成就',
    '丰功伟绩' : '丰功伟绩主要成就',
    '来源' : '来源出处',
    '在金庸小说《天龙八部》中':''
}

In [6]:
SAMPLE_LENGTH = 20

In [32]:
def save_to_disk(var):
    import pickle
    import inspect
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    var_name = [var_name for var_name, var_val in callers_local_vars if var_val is var][0]
    
    time_id = time.strftime("_%m_%d_%H_%M", time.localtime())
    
    with open(var_name + time_id + ".bin","wb") as f:
        pickle.dump(var,f)
    print(var_name + time_id + ".bin")

# 读取预解空间

In [8]:
with open("final_test_cache_07_19_17_35.bin","rb") as f:
    final_test_cache = pickle.load(f)

# 问题求解器

## 单跳问题

### 读取模型

In [9]:
from keras.models import load_model

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
one_MLP = load_model("final_one_MLP.bin")

In [11]:
with open("one_jump_clf_07_20_12_41.bin","rb") as f:
    one_jump_clf = pickle.load(f)

In [12]:
def one_jump_feature(one_jump_feature_ans,one_jump_feature_ss,one_jump_feature_label = []):
    
    """
    raw  value 定义为 (roots,edge,ans_node)
    """
    
    anses = []
    tmp_x = []
    one_jump_raw = {}
    
    fff,raw = one_jump_solver(one_jump_feature_ans,one_jump_feature_ss)
    
    for key in fff:
        anses.append(key)
        tmp_x.append(fff[key])

    try:
        tmp_x = preprocessing.scale(tmp_x)

        one_log_y = one_jump_clf.predict_proba(tmp_x)[:,1]
        one_mlp_y = one_MLP.predict(tmp_x).reshape(-1)
        one_mlp_y = one_mlp_y/one_mlp_y.max()

        #test_one_y = 0.8*test_one_log_y + 0.2*test_one_mlp_y
        y = 0.85*one_log_y + 0.15*one_mlp_y
        #y = ((np.array(tmp_x) + 0.0001)*[1,1,1,1]).sum(axis = 1)
        
        one_jump_score = y.max()
        one_jump_pre = [x for x,z in zip(anses,y) if z == one_jump_score]
        
        for high in one_jump_pre:
            one_jump_raw[high] = raw[high]
        
        one_jump_vec = fff[one_jump_pre[0]]

        #one_jump_pre = pre_clean(one_jump_pre)
        
        if one_jump_feature_label:
            one_jump_f1 = f1(pre_clean(one_jump_pre),pre_clean(one_jump_feature_label))
        else:
            one_jump_f1 = 0

        

    except ValueError:
        one_jump_pre = []
        one_jump_f1 = 0
        one_jump_vec = np.zeros(7)
        one_jump_score = 0
    
    return one_jump_pre,one_jump_raw,one_jump_f1,one_jump_vec,one_jump_score

## 双跳问题

### 读取模型

In [13]:
# tow_jump_model__07_11_13_23.model
with open("two_jump_clf_07_14_14_57.bin","rb") as f:
    two_jump_clf = pickle.load(f)

In [14]:
def two_jump_feature(two_jump_feature_ans,two_jump_feature_ss,two_jump_feature_label = []):
    
    """
    raw value 定义为 (roots,first_edge,brige_node,second_edge,ans_node) 
    """
    
    anses = []
    tmp_x = []
    two_jump_raw = {}

    fff,raw = two_jump_solver(two_jump_feature_ans,two_jump_feature_ss)
    aaa = down_to_low(fff)
    for key in aaa:
        anses.append(key)
        tmp_x.append(fff[key])

    try:
        tmp_x = preprocessing.scale(tmp_x)

        y = two_jump_clf.predict_proba(tmp_x)
        
        two_jump_score = y[:,1].max()
        
        two_jump_pre = [x for x,z in zip(anses,y[:,1]) if z == two_jump_score]
        for high in two_jump_pre:
            two_jump_raw[high] = raw[high]
        
        two_jump_vec = fff[two_jump_pre[0]]

        #two_jump_pre = pre_clean(two_jump_pre)
        
        if two_jump_feature_label:
            two_jump_f1 = f1(pre_clean(two_jump_pre),pre_clean(two_jump_feature_label))
        else:
            two_jump_f1 = 0

        

    except ValueError:
        two_jump_pre = []
        two_jump_f1 = 0
        two_jump_vec = np.zeros(7)
        two_jump_score = 0
    
    return two_jump_pre,two_jump_raw,two_jump_f1,two_jump_vec,two_jump_score

# 构造数量和相似度特征

## 构造特征用函数

In [15]:
with open("scaler_07_18_18_30.bin","rb") as f:
    scaler = pickle.load(f)

In [16]:
def seq_to_feature(seq_to_feature_ss,seq_to_feature_ans):

    #pattern = "(" + "|".join(stop_words)+")"
    #seq = re.sub(pattern,"",seq_to_feature_ss)
    #print(seq_to_feature_ss)
    
    seq = seq_to_feature_ss
    
    for key in SYNONYM_MAP:
        if key in seq_to_feature_ss:
            seq = seq.replace(key,SYNONYM_MAP[key])
    
                   
    ans = seq_to_feature_ans

    final_feature = []
    final_feature.append(q_index)
    final_feature.append(one_jump_feature(ans,seq,[]))
    final_feature.append(two_jump_feature(ans,seq,[]))
    
    return final_feature

In [17]:
def feature_for_cnn(final_feature,final_feature_scaler = scaler):
    index,one,two = final_feature
    #ss = df["question"][index]
    this_feature = []
    pattern = re.compile("[\"“”‘’''\<\>]")
    
    pre,path,F1,feature,score = two
    for key in two[1]:
        char = ""
        line = two[1][key]
        prop = line[1] + line[3]
        if len(line) > 5:
            roots = (line[0][0],line[5])
        else:
            roots = (line[0][0])
        char = "".join(roots) + line[1] + line[3]        
        this_feature.append((pattern.sub("",char),pattern.sub("",prop),ss,F1))

    pre,path,F1,feature,score = one
    char = ""
    prop = ""
    for key in one[1]:
        path = one[1][key]["path"]
        for line in path:
            tmp = line[0] + line[2]
        if tmp not in char:
            char += tmp 
            prop += line[2] 
            this_feature.append((pattern.sub("",char),pattern.sub("",prop),ss,F1))
    
    test_pre = []
    test_que = []
    test_ans = []
    for row in this_feature:
        pre,prop,que,key = row

        tmp_pre = np.array(seq_to_vec(pre,SAMPLE_LENGTH))
        tmp_que = np.array(seq_to_vec(que,SAMPLE_LENGTH))
        test_pre.append(tmp_pre)
        test_que.append(tmp_que)
        test_ans.append(key)
    test_pre = np.array(test_pre)
    test_que = np.array(test_que)
    try:
        t_test_pre = final_feature_scaler.transform(test_pre.reshape(-1,64)).reshape(-1,SAMPLE_LENGTH,64)
        t_test_que = final_feature_scaler.transform(test_que.reshape(-1,64)).reshape(-1,SAMPLE_LENGTH,64)
    
        test_x = [t_test_pre,t_test_que] * 5
    except ValueError:
        return [],[]
    return test_x,test_ans

## 构造特征

In [18]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format("file.txt", binary=False,encoding="utf8")

In [19]:
question = open("task4coqa_test.questions","rb").read().decode("utf8").split("\r\n")
total_test_feature = [] 
for num,ss in enumerate(question,0):
    ss = re.sub("(q\d{1,4}:|？)","",ss)
    print("-"*130)
    print(num,ss)
    q_index = num
    #ans = get_ans(ss,10)
    ans = final_test_cache[num][1]
    brige = seq_to_feature(ss,ans)
    total_test_feature.append((num,brige))

----------------------------------------------------------------------------------------------------------------------------------
0 ﻿德国有哪些著名的汽车品牌
----------------------------------------------------------------------------------------------------------------------------------
1 微软公司有什么软件产品
----------------------------------------------------------------------------------------------------------------------------------
2 美国总统奥巴马出生在什么地方
----------------------------------------------------------------------------------------------------------------------------------
3 麻省理工学院的创建者是
----------------------------------------------------------------------------------------------------------------------------------
4 舒伯特是哪国人
----------------------------------------------------------------------------------------------------------------------------------
5 "商鞅变法"发生于哪个时代
-------------------------------------------------------------------------------------------------------------------------------

KeyboardInterrupt: 

### 或者读取total_test_feature缓存

In [21]:
with open("total_test_feature_07_20_13_11.bin","rb") as f:
    total_test_feature = pickle.load(f)

In [22]:
def seq_to_vec(seq_to_vec_seq,length = SAMPLE_LENGTH):
    x = []
    for row in seq_to_vec_seq[:length]:
        #print(row)
        if row in model:
            x.append(model[row.upper()])

    for i in range(length - len(x)):
        x.append(model["<PAD>"])
    return x

### 读取CNN模型

In [23]:
sim = load_model("final_cnn.bin")

### 读取字向量

In [24]:
test_fff_feature = []
for num,brige in total_test_feature:
    
    tmp_feature = []
    #brige = (num,one,two,mul)
    num,one,two = brige
    
    test_x,test_ans = feature_for_cnn(brige)
    if not test_x:
        test_fff_feature.append(np.zeros(16))
        continue
    else:
        #submit.append(())
        pass
    test_y = sim.predict(test_x)[:,0]
    
    try:
        one_pre_score = test_y[[x in one[0] for x in test_ans]].max()
    except ValueError :
        one_pre_score = 0
    try:
        two_pre_score = test_y[[x in two[0] for x in test_ans]].max()
    except ValueError:
        two_pre_score = 0
    tmp_feature.extend((one_pre_score,two_pre_score))
    #这里一定要回到分类器，不然会丢失很多东西
    # 但也有可能有好处
   
    tmp_feature.extend(one[3])
    tmp_feature.extend(two[3])
    
    tmp_feature.extend((two[4],one[4]))
    
    test_fff_feature.append(tmp_feature)

#  感知机分类

In [25]:
with open("scaler_2_07_20_12_56.bin","rb") as f:
    scaler_2 = pickle.load(f)

In [26]:
submit_y = scaler_2.transform(np.array(test_fff_feature))

In [27]:
final_model = load_model("final_model_07_18.bin")

In [28]:
nnn = final_model.predict(np.array(submit_y))

In [31]:
save_to_disk(nnn)

nnn_07_20_17_18.bin


# 生成感知机答案

In [29]:
submit = []
for num,brige in total_test_feature:
    index,one,two = brige
    ans_list = [one,two]
    aaa = nnn.argmax(axis = 1)[num]
    #print(aaa)
    try:
        submit.append(ans_list[int(aaa)][0])
    except TypeError:
        submit.append([])

In [30]:
with open("result_ss_new.txt","wb") as f:
    for line in submit[:-1]:
        line = "\t".join(line) + "\r\n"
        f.write(line.encode("utf8"))